# FINM 250 Homework 4
### TA Solutions

## Part 3

### 1

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import sys

sys.path.append("../cmds")
from utils import (
    calc_univariate_regression,
    calc_iterative_regression,
    calc_performance_metrics
)


# Plotting settings.
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["font.family"] = "serif"
plt.style.use("ggplot")

# Pandas settings.
pd.set_option("display.float_format", lambda x: "{:.4f}".format(x))

ADJ = 12

from functools import partial

get_data = partial(
    pd.read_excel, "../data/gmo_analysis_data.xlsx", index_col=0, parse_dates=[0]
)

signals = get_data(sheet_name="signals")
returns = get_data(sheet_name="returns (total)")
spy = returns[['SPY']]
risk_free = get_data(sheet_name="risk-free rate")

In [4]:
signals_lag = signals.shift(1).dropna()
spy = spy.loc[signals_lag.index]

# Expanding.
forecast_df = spy.expanding().mean().shift(1).dropna()
forecast_df.columns = ['Mean']

display(forecast_df.head())

,Mean
Date,
1997-01-31,-0.0238
1997-02-28,0.0190
1997-03-31,0.0159
1997-04-30,0.0009
1997-05-31,0.0132


In [6]:

# Multi-variate regression.
multi_regr = sm.OLS(spy, sm.add_constant(signals_lag)).fit()
print(multi_regr.summary())


                            OLS Regression Results                            
Dep. Variable:                    SPY   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.502
Date:                Tue, 11 Jul 2023   Prob (F-statistic):              0.214
Time:                        04:23:01   Log-Likelihood:                 539.75
No. Observations:                 319   AIC:                            -1071.
Df Residuals:                     315   BIC:                            -1056.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0145      0.020     -0.719      0.4

In [8]:


forecast_df['Multi'] = multi_regr.params[0] + multi_regr.params[1] * signals['DP'] + multi_regr.params[2] * signals['EP']

print('-'*50)
print('Multivariate Regression Summary')
display(multi_regr.params)

print('-'*50)
print(f'R^2: {multi_regr.rsquared:.4f}')

forecast_df.dropna(inplace=True)
forecast_df['Actuals'] = spy['SPY']
forecast_df['Risk-Free'] = risk_free['US3M']


--------------------------------------------------
Multivariate Regression Summary


const   -0.0145
DP       0.0092
EP       0.0021
US10Y   -0.0010
dtype: float64

--------------------------------------------------
R^2: 0.0141


### 2

In [ ]:
forecast_wt = forecast_df * 100
forecast_model = forecast_wt[['DP', 'EP', 'Multi']]
forecast_model = forecast_model.multiply(forecast_df['Actuals'], axis=0)
forecast_model['Actuals'] = forecast_df['Actuals']

display(calc_performance_metrics(forecast_model).T)

# Calculate market stats.
calc_iterative_regression(forecast_model, forecast_model[['Actuals']], one_to_many=True).iloc[:-1, :]

### 3

In [ ]:
forecast_model['Risk-Free'] = forecast_df['Risk-Free']

forecast_subsample = forecast_model.loc['2000':'2011']
display(calc_performance_metrics(forecast_subsample).T)

fig, ax = plt.subplots(figsize=(8, 5))
forecast_subsample_cum = (forecast_subsample + 1).cumprod() - 1
forecast_subsample_cum.plot(ax=ax)

ax.set_title('Cumulative Returns')
ax.set_ylabel('Cumulative Return')
ax.set_xlabel('Date')
fig.tight_layout()

Short term bonds outperform all by the earnings-to-price ratio. However, the comparison is also a little unfair given that we are including the 2008 financial crisis, prior to 2008, all strategies except for SPY either outperform or are on par/slightly below the cumulative return of 3-month T-bills.

### 4

In [ ]:
# Count negative forecasted returns.
neg_count = (forecast_df < 0).sum().to_frame('Count')
neg_count['Percentage'] = neg_count / len(forecast_df)
neg_count

### 5

In [ ]:
# Check risk metrics for GMWAX. 
display(calc_performance_metrics(returns[['GMWAX']]).T)
display(calc_univariate_regression(returns['GMWAX'], returns['SPY']))

For $r^X$; yes. Given that for all of the models we tested, they have very high market betas. So they are just loading up on market risk (by construction), so they do have a worse Sharpe Ratio, higher kurtosis, and worse risk metrics overall. I would say the EP performs the best out of all of them. 

Looking at GMWAX, I don't think it is necessarily risk that makes it worse, but rather it is just worse than SPY across the board, though it has a better max drawdown. Finally, it has a somewhat high market beta, at 0.55.